In [355]:
RAW_DATA = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\AAA.csv'
out_path = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\520mV.HGA Lifecycle.520mV9999999999.xlsx'

In [356]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference

In [357]:
def disp(var1,var2,var3,var4):
    cal = np.where(var1!=var2,0,np.where(var1!=var3,1,np.where(abs(var4)>=20,1,0)))
    return(cal)

def disp1(var1,var2,var3,var4):
    cal = np.where(var1!=var2,0,np.where(var3!=var4,1,0))
    return(cal)

def disp2(var1,var2):
    #cald = (abs(var2)>=20) & (var1 == 1)
    cal = np.where((abs(var2)>=20) & (var1 == 1),'F','S')
    return(cal)

In [358]:
df = pd.read_csv(RAW_DATA, header=[2])
#df = pd.read_csv(RAW_DATA, header=[2], sep=r"[| ^]", engine='python')

#df = df[['Hd','Cycles','Part ID','Test Time','Test','Date','Time']]
df = df.iloc[:,[0,1,4,5,6,7,26,30]]
#cols = {0:'Time Stamp(S)',30:'Resistance (Ohms)'}
#df.rename(columns=cols, inplace=True)
df.columns = ['Hd','Part ID','Test Time','Test','Date','Time','TimeStamp (s)','Resistance (Ohm)']
df = df.sort_values(['Hd','Date','Time'])
df['Total Stress-Hrs'] = np.where(df['Hd']!=df['Hd'].shift(),0,df['TimeStamp (s)'])/3600
grouped_df = df.groupby(['Part ID']).first()
df = pd.merge(df, grouped_df, on=['Part ID'] , suffixes=('','_REF'))
df['Delta_Res(%)'] = (df['Resistance (Ohm)']/df['Resistance (Ohm)_REF'] - 1)*100
df['P/F_Status'] = disp(df['Part ID'],df['Part ID'].shift(),df['Part ID'].shift(-1),df['Delta_Res(%)'])
df['P/F_Flag'] = disp1(df['Part ID'],df['Part ID'].shift(),df['P/F_Status'],df['P/F_Status'].shift())
df['F/S'] = disp2(df['P/F_Flag'],df['Delta_Res(%)'])
df['Stress(mV)'] = df['Test'].str[14:17]
df = df[['Hd','Part ID','Test Time','Test','Date','Time','TimeStamp (s)','Resistance (Ohm)','Total Stress-Hrs','Resistance (Ohm)_REF','Delta_Res(%)','P/F_Status','P/F_Flag','F/S','Stress(mV)']]

dy = df.loc[df['P/F_Flag'] == 1]
table = pd.pivot_table(dy, values='Part ID', index=['F/S','Total Stress-Hrs'], columns=[], aggfunc='count', margins=False)

In [359]:
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='ALL', index=None)
table.to_excel(writer, sheet_name='SUM')
dy.to_excel(writer, sheet_name='TTF')
writer.save()